# **This notebook aims to extract data from a correctly formatted CSV file and adapt it to the pangeo-fish format**

### **Necessary imports**
___

In [ ]:
import pandas as pd
import numpy as np
from data_conversion import extract_tagging_events
from data_conversion import create_metadata_file
from data_conversion import extract_name
from data_conversion import format_date
from data_conversion import extract_DST
from data_conversion import convert_to_utc_with_formatting

import csv
import os
from tqdm import tqdm

In [ ]:
remote = True

In [ ]:
### Test with the tag NO_A12667
### These two paths will be used as an example to see if the full data extraction works correctly
tag_id = "DK_A10627"
csv_path = f"s3://gfts-ifremer/tags/bargip/raw/{tag_id}.CSV"  # Path to the raw csv file, where the code will extract data from. Update with yours to adapt
destination = f"s3://gfts-ifremer/tags/bargip/clean_demo/{tag_id}/"  # Folder where you want to write your the different files. Update with yours to adapt

if not remote:
    os.makedirs(destination, exist_ok=True)

___
### 1. **Extracting the tagging events**
In this section, we try to test and compare how to extract the necessary information for the tagging events (i.e., time and position for release, fish death, (recapture?))
___

See below the steps that the extract_DST function does: 

- **Purpose**:
  - Extracts releasing date, presumed fish death date, and fish release/recapture positions from a CSV file.

- **Initialization**:
  - Initializes variables for storing dates (`release_date`, `fish_death`) and coordinates (`lon`, `lat`).

- **Processing CSV**:
  - Opens the CSV file and iterates through each line.

- **Data Extraction**:
  - Extracts releasing date and presumed fish death date.
  - Formats latitude and longitude coordinates for fish release/recapture positions.

- **DataFrame Creation**:
  - Constructs a DataFrame with event names, dates, longitude, and latitude.
  - Returns the DataFrame containing tagging events data.

In [ ]:
import s3fs

s3 = s3fs.S3FileSystem(
    anon=False,
    client_kwargs={
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
    },
)

In [ ]:
### See the function tagging_events in the file data_conversion.py for further information
tagging_events = extract_tagging_events(
    csv_path, time_zone="Europe/Paris", remote=remote
)
te_save_path = destination + "tagging_events.csv"
# tagging_events.to_csv(te_save_path,index=False)
tagging_events

___
### 2. **Creating the metadata JSON file**
In this section, we try to test and compare how to extract the necessary information for the metatdat.json file.  
___
- **Purpose**:
  - Creates a metadata JSON file based on provided data path and destination path.

- **Initialization**:
  - Retrieves tag name from the provided file path using a helper function.
  - Initializes metadata with tag ID, scientific name, common name, and project information.

- **Metadata Construction**:
  - Constructs a dictionary (`metadata`) containing tag ID, scientific name ("Dicentrarchus labrax"), common name ("European seabass"), and project name ("BARGIP").

- **File Writing**:
  - Specifies the filename as "metadata.json" and constructs the full destination path.
  - Writes the metadata dictionary to a JSON file at the destination path.

- **Result**:
  - No return value; a metadata JSON file is created at the specified destination path.

In [ ]:
### See data_conversion.py for more information about create_metadata_file function
create_metadata_file(csv_path, destination, remote=True)

___
### 3. **Creating the dst.csv file**
In this section, we will create the dst file that contains the pressure, temperature and time data.  
See below the steps that the extract_DST function does:  
___

- **Opening the CSV File**:
  - Takes a file path to a CSV file containing time series data.
  - Opens the CSV file using the `csv.reader` object.
  
- **Iterating Through CSV Rows**:
  - Iterates through each row of the CSV file.
  
- **Extracting Tag ID**:
  - Extracts the tag ID from the file path using the `extract_name` function (not provided).
  
- **Finding Target Line**:
  - Searches for the line that contains the headers for the data of interest ("Date/Time Stamp", "Pressure", "Temp").
  
- **Reading Data**:
  - Once the target line is found, starts reading data rows.
  
- **Formatting Date and Time**:
  - Formats the date and time column using the `convert_to_utc_with_formatting` function.
  - Converts the local time to UTC time based on the specified time zone.
  
- **Converting Data Types**:
  - Converts the pressure and temperature data from strings to `numpy.float64` for numerical analysis.
  
- **Storing Data**:
  - Stores the formatted data into a list for further processing.
  
- **Creating DataFrame**:
  - After reading all data rows, converts the list of data into a Pandas DataFrame with columns ['time', 'pressure', 'temperature'].
  
- **Completeness Check**:
  - Checks if the number of data points extracted matches the expected length.
  - If they match, indicates completion; otherwise, suggests potential incompleteness.
  
- **Returning DataFrame**:
  - Finally, returns the DataFrame containing the extracted data.

This function primarily focuses on extracting time, pressure, and temperature data from a CSV file, converting the date and time to UTC time, and formatting the data for analysis. Check the function in the file data_conversion.py for further information.

In [ ]:
csv_path

In [ ]:
time_zone = "Europe/Paris"
dst = extract_DST(csv_path, time_zone, remote=True)
dst

In [ ]:
dst_save_path = destination + "dst.csv"
dst.to_csv(dst_save_path, index=True)

___
### 4. **Convert and format everything under the raw_test folder to the cleaned folder**
This section has test purpose to see if it's easy and works correctly for the different tags in the **raw_test** folder.  
Afterwards, the purpose is to do the same operation on the **raw** folder
___
#### Explenation of the code below :
- **Folders and Time Zone Setup**:
  - Defines folders (`raw_folder`, `destination_folder`) and time zone (`time_zone`).

- **Destination Folder Creation**:
  - Checks if the destination folder exists; if not, creates it.

- **Processing Raw Data**:
  - Iterates through raw files in the raw folder.
  - Extracts tag ID and constructs destination paths.
  - Creates tag-specific folders if they don't exist.
  - Extracts tagging events and DST data from raw files.
  - Saves extracted data to CSV files in respective tag folders.
  - Creates metadata files for each raw file.

- **Handling Incorrect Raw Folder**:
  - Prints a message if the raw folder doesn't exist.

In [ ]:
%%time

### Local
if not remote:
    raw_folder = "../../all_raw/"  # Folder name to explore
    destination_folder = "../../all_cleaned/"
    time_zone = "Europe/Paris"

    if not os.path.exists(destination_folder):
        os.mkdir(destination_folder)

    # Check if the folder exists
    if os.path.exists(raw_folder):
        # Get list of files to iterate through
        files = [
            f
            for f in os.listdir(raw_folder)
            if os.path.isfile(os.path.join(raw_folder, f))
        ]

        # Wrap files list with tqdm for progress bar
        for file_name in tqdm(files, desc="Processing files"):
            raw_file = os.path.join(raw_folder, file_name)

            # Extract filename without extension
            tag_id = extract_name(raw_file)
            destination_path = os.path.join(destination_folder, tag_id)

            # Check if the folder for the tag exists, if not, create it
            if not os.path.exists(destination_path):
                # print("Creating folder for tag:", tag_id)
                os.mkdir(destination_path)

            ### Extracting tagging events from raw file
            tag_events = extract_tagging_events(raw_file)
            tagging_events_path = os.path.join(destination_path, "tagging_events.csv")
            tag_events.to_csv(
                tagging_events_path, index=False
            )  ### Saving them at the right path

            ### Extracting DST from raw file
            tag_dst = extract_DST(raw_file, time_zone)
            dst_path = os.path.join(destination_path, "dst.csv")
            tag_dst.to_csv(dst_path, index=False)  ### Saving them at the right path

            ###Creating metadata files
            # print("creating_metadata")
            create_metadata_file(raw_file, destination_path)

    # else:
    # print("Wrong folder for raw files")

In [ ]:
csv_path

In [ ]:
%%time
### Remote
if remote:
    s3 = s3fs.S3FileSystem(
        anon=False,
        client_kwargs={
            "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",  # S3 endpoint for OVH
        },
    )
    raw_folder = "s3://gfts-ifremer/tags/bargip/raw"  # Folder name to explore
    destination_folder = "s3://gfts-ifremer/tags/bargip/clean_demo/"
    time_zone = "Europe/Paris"

    # if not os.path.exists(destination_folder):
    #     os.mkdir(destination_folder)

    # Check if the folder exists
    # if os.path.exists(raw_folder):

    # Get list of files to iterate through
    tag_ids = [
        tag_id.replace("gfts-ifremer/tags/bargip/raw/", "").replace(".CSV", "")
        for tag_id in s3.ls("gfts-ifremer/tags/bargip/raw")
    ]

    # Wrap files list with tqdm for progress bar
    for tag_id in tqdm(tag_ids, desc="Processing files"):
        print(tag_id)
        try:
            file_path = f"{raw_folder}/{tag_id}.CSV"

            # # Extract filename without extension
            destination = f"{destination_folder}{tag_id}"

            ### Extracting tagging events from raw file
            tag_events = extract_tagging_events(file_path, remote=True)
            te_save_path = f"{destination}/tagging_events.csv"
            tag_events.to_csv(te_save_path, index=False)

            # ### Extracting DST from raw file
            tag_dst = extract_DST(file_path, time_zone, remote=True)
            dst_save_path = f"{destination}/dst.csv"
            tag_dst.to_csv(dst_save_path, index=False)

            ###Creating metadata files
            # print("creating_metadata")
            create_metadata_file(file_path, destination, remote=True)
        except Exception as e:
            print(f"Error for {tag_id}")
            print(e)
# else:
# print("Wrong folder for raw files")

In [ ]:
toto = pd.read_csv("s3://gfts-ifremer/tags/bargip/cleaned/DK_A10625/dst.csv")

In [ ]:
pd.read_csv("s3://gfts-ifremer/tags/bargip/raw/DK_A10625.CSV", sep=";")

In [ ]:
# Run this cell if you computed the tags locally and you want to put all tags from local to the bucket

import s3fs

s3 = s3fs.S3FileSystem(
    anon=False,
    client_kwargs={
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
    },
)

if not remote:
    s3.put("../../all_cleaned/", "gfts-ifremer/tags/bargip/cleaned", recursive=True)